# Transformation données avec mediapipe

In [1]:
import numpy as np
import pandas as pd
import mediapipe as mp
import os
import cv2
from PIL import Image
import csv

Le jeu de donnée utilisé est composé de 72 000 photos (200*200 pixel) classées dans 24 dossiers par signes représentés

In [17]:
# on créer une liste des dossiers à consulter et on élimine 3 catégories (del, nothing et space)
label = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']
#on parametre la bibliothèque hands de mediapipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.3)

for l in label:   
    dataset_directory_prefix = 'C:/Users/Maxime.ALTER/ProjetIA/ProjetIA/Dev_tib/photos_asl/asl_alphabet_train/asl_alphabet_train/'
    #dataset_directory_prefix = r'C:/Users/Panah/OneDrive/Bureau/data/asl_alphabet_train/asl_alphabet_train/'
    
    for dirname, _, filenames in os.walk(dataset_directory_prefix + l):
        
        for filename in filenames:
            
            #on viens lire les images dans leurs dossier respectifs
            pathname = os.path.join(dirname, filename)
            image = cv2.imread(pathname)
            image = cv2.flip(image, 1)         
            #on process l'image avec la librairie hand
            results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            #On stocke la première lettre du nom du fichier qui servira pour créer le label
            coord_list = [filename[0]]
            target = []
            
            #On récupère l'indice de la lettre dans notre liste de label, qui deviendra notre target
            for letter in coord_list : 
                if letter in label:
                    target.append(label.index(letter))
                         

            if results.multi_hand_landmarks:                             
                
                for hand_no, hand_landmarks in enumerate(results.multi_hand_landmarks):
                    #print(f'HAND NUMBER: {hand_no+1}')
                    #print('-----------------------')
                    
                    #On parcours les 21 landmarks pour récupèrer les valeurs x, y et z et les stocker dans notre liste                    
                    for i in range(21):
                        name = mp_hands.HandLandmark(i).name
                        x = hand_landmarks.landmark[mp_hands.HandLandmark(i).value].x
                        y = hand_landmarks.landmark[mp_hands.HandLandmark(i).value].y
                        z = hand_landmarks.landmark[mp_hands.HandLandmark(i).value].z
                        
                        #coord_list.append(name)
                        target.append(x)
                        target.append(y)
                        target.append(z)
                    
                    #A chaque image, on écrit une ligne dans un csv contenant la target suivis coordonnées x, y et z des 21 landmarks
                    
                    with open(r'C:/Users/Maxime.ALTER/ProjetIA/ProjetIA/Dev_max/landmark.csv', 'a', newline='') as csvfile:
                        writer = csv.writer(csvfile)
                        writer.writerow(target)